In [ ]:
import pandas as pd
import pyabf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

#import data from spreadsheet 
#This has many columns with redundant data for this project and was made for people to read in excel
ABF_data = pd.read_excel(*Path to file here*)



In [ ]:
#df[df.eq("R347P").any(axis=5)]
#Pull data only with the R347P mutation
R347P_data = ABF_data.loc[ABF_data['Mutation']=='R347P']
del(ABF_data)
#Pull ER data, ER data referes to spinability recording
R347P_data = R347P_data.loc[R347P_data['Type'] =='ER']
#And pull data where DMSO is applied to the culture
R347P_data_dmso=R347P_data.loc[R347P_data["Drugs"]=="DMSO"]
#Pull data where '3VX' is applied
R347P_data_vx=R347P_data.loc[R347P_data["Drugs"]=="VX445, VX661, VX770"]

#Create the train/test split for DMSO data
R347P_DMSO_train, R347P_DMSO_test = train_test_split(R347P_data_dmso, test_size=0.2)
#Create the train/test split for the 3VX data
R347P_vx_train, R347P_vx_test = train_test_split(R347P_data_vx, test_size=0.2)
#free up memory
del(R347P_data)
#create the train dataset
train = pd.concat([R347P_DMSO_train, R347P_vx_train])

#Create the test dataset
test = pd.concat([R347P_DMSO_test, R347P_vx_test])

#Resetting indext to allow for innmumeration later
train =  train.reset_index()
test =  test.reset_index()

#Free up memory 
del(R347P_DMSO_train)
del(R347P_DMSO_test)
del(R347P_vx_train)
del(R347P_vx_test)

In [ ]:
#### This section populates the train array

#Initalise array to write ABF files to 
ABF_train_as_array = []
#File locations
abf_loc = *Location Here*


#for loop to cycle through train array and extract ABF file names
for index, row in train.iterrows():
    #Loading in the ABD files
    temp_array = pyabf.ABF(abf_loc+str(row['Recording name']) + ".abf")
    #Specify which channel to call
    temp_array.setSweep(sweepNumber=0, channel=0)
    #ABF files load y axis (the current) and x axis (time for each array poin), X is not relevant in this case as it can be claulated via the y axis
    temp_y = temp_array.sweepY
    
    #The first 2-10 seconds are not useful in this data. Withdraw time is the time the pipette withdrawal starts
    withdraw_start = row['Withdraw Time']
    #Time per array element is given here from the ABF data
    time_per_array = temp_array.sweepX[1] 
    #The point in the array where the withdawal startes is calulaed by the start time (seconds)/Time per array element     
    withdraw_start_point = round(withdraw_start/time_per_array)
    #Normalise the data as the current can vary (with pipette resistant and bath electrode), this also allow normaliseation for the shape of the curve
    norm_y = (temp_y-np.min(temp_y))/(temp_y[withdraw_start_point]-np.min(temp_y))
    #Trim useless data from array
    norm_y = norm_y[withdraw_start_point:]
    # downsampling all arrays so they are the same size on the time axis allowing for analysis of the shape
    #This is also done as there is a large variation in the time of the recordings which makes using a CNN more complicated
    #shortest array is length 36320
    window_size = round(len(norm_y)/36320)
    #Find the remainder and add on as paddding, to allow for uniform downsampling resulting in uniform array sizes
    padding = len(norm_y)%window_size
    norm_y.resize((len(norm_y+padding)))
    #array to assign re-sized arrays to
    temp_norm_y = []
    #end point is to allow for the last pass of the median window
    end = int(len(norm_y)-window_size)
    #shift the window the length of the window each time
    for i in range (0,end,window_size+1):
        #caclulte end of median window
        window_end = i+window_size
        #median filter is used as traces can suffer from noise, the median filter will reduce noise and array size
        median_val = np.median(norm_y[i:window_end])
        #append the median to the final array
        temp_norm_y.append(median_val)
    #convert to array
    temp_norm_y = np.asarray(temp_norm_y) 
    #the resizing does not work perfectly! I think this is due to rounding and calculation of the window size. 
    #This needs fixing but the results are close, so for not cropping, or zero padding. This needs fixing but is not urgent
    temp_norm_y.resize(36320)
    #convert to array
    temp_array_np = np.array(temp_norm_y)
    #add to final data array
    ABF_train_as_array.append(temp_norm_y)
    

#print(ABF_train_as_array)

In [ ]:
#Initalise array to write ABF files to 
ABF_test_as_array = []
#File locations
abf_loc = 'SICM_files/'


#for loop to cycle through train array and extract ABF file names
for index, row in test.iterrows():
    #Loading in the ABD files
    temp_array = pyabf.ABF(abf_loc+str(row['Recording name']) + ".abf")
    #Specify which channel to call
    temp_array.setSweep(sweepNumber=0, channel=0)
    #ABF files load y axis (the current) and x axis (time for each array poin), X is not relevant in this case as it can be claulated via the y axis
    temp_y = temp_array.sweepY
    
    #The first 2-10 seconds are not useful in this data. Withdraw time is the time the pipette withdrawal starts
    withdraw_start = row['Withdraw Time']
    #Time per array element is given here from the ABF data
    time_per_array = temp_array.sweepX[1] 
    #The point in the array where the withdawal startes is calulaed by the start time (seconds)/Time per array element     
    withdraw_start_point = round(withdraw_start/time_per_array)
    #Normalise the data as the current can vary (with pipette resistant and bath electrode), this also allow normaliseation for the shape of the curve
    norm_y = (temp_y-np.min(temp_y))/(temp_y[withdraw_start_point]-np.min(temp_y))
    #Trim useless data from array
    norm_y = norm_y[withdraw_start_point:]
    # downsampling all arrays so they are the same size on the time axis allowing for analysis of the shape
    #This is also done as there is a large variation in the time of the recordings which makes using a CNN more complicated
    #shortest array is length 36320
    window_size = round(len(norm_y)/36320)
    #Find the remainder and add on as paddding, to allow for uniform downsampling resulting in uniform array sizes
    padding = len(norm_y)%window_size
    norm_y.resize((len(norm_y+padding)))
    #array to assign re-sized arrays to
    temp_norm_y = []
    #end point is to allow for the last pass of the median window
    end = int(len(norm_y)-window_size)
    #shift the window the length of the window each time
    for i in range (0,end,window_size+1):
        #caclulte end of median window
        window_end = i+window_size
        #median filter is used as traces can suffer from noise, the median filter will reduce noise and array size
        median_val = np.median(norm_y[i:window_end])
        #append the median to the final array
        temp_norm_y.append(median_val)
    #convert to array
    temp_norm_y = np.asarray(temp_norm_y) 
    #the resizing does not work perfectly! I think this is due to rounding and calculation of the window size. 
    #This needs fixing but the results are close, so for not cropping, or zero padding. This needs fixing but is not urgent
    temp_norm_y.resize(36320)
    #convert to array
    temp_array_np = np.array(temp_norm_y)
    #add to final data array
    ABF_test_as_array.append(temp_norm_y)
    


In [ ]:
#This part of the script writes the data to CSV files 

import csv

with open('train_data.csv', 'w') as f:
    
    # Create a CSV writer object that will write to the file 'f'
    csv_writer = csv.writer(f)
    
    # Write the field names (column headers) to the first row of the CSV file
    #csv_writer.writerow(fields)
    
    # Write all of the rows of data to the CSV file
    csv_writer.writerows(ABF_train_as_array)
# close the file
f.close()


# open the file in the write mode
with open('train_labels.csv', 'w') as f:

    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    writer.writerows(train['Drugs'])

# close the file
f.close()



In [ ]:
#convert dtype for data
train_ABF_as_np_array = np.asarray(ABF_train_as_array,dtype=object)
test_ABF_as_np_array = np.asarray(ABF_test_as_array,dtype=object)
#find minimum value, This is now earlier in the script
#min_l = inf
#find smllest array length in the training data
#for l in train_ABF_as_np_array:
 #   Length = len(l)

  #  if Length < max_l:
   #     min_l = Length



#Length = inf

#find smllest array length in the test data
#for l in test_ABF_as_np_array:
 #   Length = len(l)
    #print(Length)
  #  if Length < max_l:
   #     min_l = Length
#print('Max length')
#print(max_l)
#cycle 
#count =0
#for l in train_ABF_as_np_array:

 #   temp = np.array(l)

  #  train_ABF_as_np_array[count] = temp

   # count = count + 1
 
 #count = 0    
#for l in test_ABF_as_np_array:

 #   Length = max_l - len(l)

  #  temp = np.array(l)

   # temp.resize(max_l)

    #test_ABF_as_np_array[count] = temp

    #count = count + 1    


In [ ]:
#For debugging to check arrays re the same length
#for i in train_ABF_as_np_array:
 #   print(len(i))

In [ ]:
#####This part initlises and trains the model ####
##inital CNN
input_shape= (36320,1)
#populates the label array 
labels_binary = []
#Cycles through 
for i in train['Drugs']:
    if i == 'DMSO':
        labels_binary.append(0)
    else:
        labels_binary.append(1)
#convert data type        
labels = np.asarray(labels_binary).astype(np.int32)


train_data = np.asarray(train_ABF_as_np_array).astype(np.int32)
test_data = np.asarray(test_ABF_as_np_array).astype(np.int32)

#populate test labels
labels_binary_test = []
for i in test['Drugs']:
    if i == 'DMSO':
        labels_binary_test.append(0)
    else:
        labels_binary_test.append(1)
     

#other inputs for the model 
#ASL dpeth
train_depths = np.asarray(train['ASL Depth (mm)']).astype(np.int32)
#infection
train_infection = np.asarray(train['Infected (0 no 1 day before infection 2 infected while measuring)']).astype(np.int32)
#snap length
train_snap = np.asarray(train['Snap Lengths (um)']).astype(np.int32)
#90% current reduction
train_90_current = np.asarray(train['90% current redcution point']).astype(np.int32)


#test inputs
test_depths = np.asarray(test['ASL Depth (mm)']).astype(np.int32)
test_infection = np.asarray(test['Infected (0 no 1 day before infection 2 infected while measuring)']).astype(np.int32)
test_snap = np.asarray(test['Snap Lengths (um)']).astype(np.int32)
test_90_current = np.asarray(test['90% current redcution point']).astype(np.int32)




#transpose arrays to match indicies
input_train = np.asarray([train_depths,train_infection,train_snap,train_90_current]).T

input_test = np.asarray([test_depths,test_infection,test_snap,test_90_current]).T

labels_test = np.asarray(labels_binary_test).astype(np.int32)
#val_dataset = [test_data, input_test]
#val_dataset = np.asarray(val_dataset).astype(np.int32)
#val_dataset = np.asarray(val_dataset).astype(np.int32)
#initilise column 1, this is about as big as my laptop gpu can handel
input_A = tf.keras.layers.Input(input_shape)
x = tf.keras.layers.Conv1D(32, kernel_size=1028, activation='relu')(input_A)
#x = tf.keras.layers.Dropout(0.2)(x)
#x = tf.keras.layers.Conv1D(32, kernel_size=512, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv1D(32, kernel_size=256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv1D(32, kernel_size=128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
#x = tf.keras.layers.Conv1D(32, kernel_size=64, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.2)(x)
#x = tf.keras.layers.Conv1D(32, kernel_size=32, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv1D(32, kernel_size=16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
#x = tf.keras.layers.Conv1D(32, kernel_size=8, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv1D(16, kernel_size=8, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv1D(8, kernel_size=8, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)

x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.Model(inputs=input_A, outputs=x)


#outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
#initilise column 2

input_B = tf.keras.layers.Input((4))
x1 = tf.keras.layers.Dense(32, activation='relu')(input_B)
x1 = tf.keras.layers.Dropout(0.2)(x1)
x1 = tf.keras.layers.Dense(16, activation='relu')(x1)
x1 = tf.keras.layers.Dropout(0.2)(x1)
x1 = tf.keras.layers.Dense(8, activation='relu')(x1)
x1 = tf.keras.layers.Flatten()(x1)
x1 = tf.keras.Model(inputs=input_B, outputs=x1)

#combine dense and CNN
combined = tf.keras.layers.concatenate([x.output, x1.output])

z = tf.keras.layers.Dense(2, activation="relu")(combined)
z = tf.keras.layers.Dense(16, activation="relu")(z)
z = tf.keras.layers.Dense(1, activation="sigmoid")(z)


#create model
model = tf.keras.Model([input_A, input_B], z)
#train using binary cross entropy
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#fit model
model.fit(x=[train_data, input_train], y=labels, batch_size=4, epochs=100,validation_data=([test_data, input_test],labels_test))

